# Regresión Lineal

$y=a+bx+\epsilon$

El modelo de regresión lineal asume que la relación entre una variable continua dependiente $y$ y una o más variables explicativas (independientes) $X$ es lineal (es decir, que lo podemos modelizar con una línea recta). 

Se utiliza para predecir valores dentro de un rango continuo (por ejemplo, cantidad de ventas, el precio en función de otras variables).

<img src="images/regresion_lineal.jpeg" alt="Drawing" style="width: 600px;"/>

Fuente: http://mybooksucks.com

## Prediciendo la presión sanguinea a partir de la edad...

Utilizaremos datos de medición de la presión sanguínea sistólica (medida en mm de Mercurio) para 29 personas de diferentes edades.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# df = pd.read_csv('datasets/regresion/precio_viviendas.csv')

df = pd.read_csv('datasets/regresion/edad_presion.csv', skiprows=32, usecols=[2,3])

In [ ]:
df.rename(columns={'Age': 'edad', 'Systolic blood pressure': 'presion'}, inplace=True)

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
# Vamos a intentar predecir el precio de venta (SalePrice) en función de la cantidad de superficie habitable (en pies cuadrados) (GrLivArea)

# df = df[['GrLivArea', 'SalePrice']]

In [ ]:
plt.scatter(df['edad'], df['presion'], alpha=0.2, edgecolor='black')
plt.xlabel('edad')
plt.ylabel('presion');

### División del dataset en conjunto de entrenamiento y prueba

Para entrenar cualquier modelo de aprendizaje automático, independientemente del tipo de conjunto de datos que se esté utilizando, debemos dividir el conjunto de datos en datos de entrenamiento y datos de prueba y datos de validación. Pero, ¿por qué?

Entrenar un modelo de aprendizaje automático supervisado es conceptualmente muy simple e implica el siguiente proceso de tres pasos:

**1. Enviar muestras del conjunto de datos a un modelo (inicializado)**: las muestras de su conjunto de datos se transmiten a través del modelo, lo que genera predicciones.

**2. Comparar predicciones y verdad fundamental**: las predicciones se comparan con las etiquetas verdaderas correspondientes a las muestras, lo que nos permite identificar qué tan mal funciona el modelo.

**3. Loop de mejora**: basándonos en la métrica de optimización, podemos cambiar las partes internas del modelo aquí y allá, para que (con suerte) funcione mejor durante la próxima iteración.


Cuando continúe realizando estas iteraciones, el modelo continuará mejorando, porque puede "explotar" todos los patrones espurios en su conjunto de datos.
Pero, ¿qué pasa si esos patrones espurios no están presentes en los datos del mundo real para los que generará predicciones después del entrenamiento? ¿Qué sucede si, por lo tanto, el modelo se entrena en patrones que son exclusivos del conjunto de datos de entrenamiento y no están presentes en el conjunto de datos para la inferencia ?

Entonces, dicho brevemente, tiene un problema.



* **Conjunto de entrenamiento (train set)**: es el conjunto de datos utilizados para el aprendizaje del modelo, es decir, para ajustar los hiperparámetros al modelo de aprendizaje automático.

* **Conjunto de datos de prueba (test set)**: es el conjunto de datos utilizados para proporcionar una evaluación imparcial del modelo final ajustado en el conjunto de datos de entrenamiento.

<img src="images/split_dataset.webp" alt="Drawing" style="width: 300px;"/>

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.20, random_state=33)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.scatter(df_train['edad'], df_train['presion'], alpha=0.2, edgecolor='black')
ax1.set_ylabel('presion')
ax1.set_xlabel('edad')
ax1.set_title('Datos de entrenamiento')
ax2.scatter(df_test['edad'], df_test['presion'], alpha=0.2, edgecolor='black')
ax2.set_ylabel('presion')
ax2.set_xlabel('edad')
ax2.set_title('Datos de validación');

# Pseudoinversa

Matrices que no tienen inversa:
  * det = 0
  * No cuadradas

$Y = aX$

Si quiero obtener el $a$ que minimiza el error cuadrático medio utilizando la pseudoinversa, simplemente

$\hat{a} = (X^T X)^{-1} X^T Y$

En nuestro caso,

Aunque a priori, parece que solo hay un predictor ($ x_1 $), nuestro modelo requiere un segundo (llamémoslo $ x_0 $) para permitir una intercepción con $ y $. Sin esta segunda variable, la línea que ajustamos a la gráfica tendría que pasar por el origen (0, 0). La intersección $ y $ es constante en todos los puntos, por lo que podemos establecerla igual a 1 en todos los ámbitos:

$Y = a + bX = \left[\mathbf{1}; X\right] \begin{bmatrix} a \\ b\end{bmatrix}$

##### Para nuestro ejemplo...

In [ ]:
X = np.vstack((np.ones_like(df_train['edad']), df_train['edad'])).T

In [ ]:
X.shape

In [ ]:
# Y = a + bX
a, b = np.dot(np.linalg.pinv(X), df_train['presion'])
print(f"Solución por pseudoinversa (Y=a+bX) => a={a}  |  b={b}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Datos de entrenamiento
ax1.scatter(df_train['edad'], df_train['presion'], alpha=0.2, edgecolor='black')
ax1.set_ylabel('presion')
ax1.set_xlabel('edad')
ax1.set_title('Datos de entrenamiento')

x1_min, x1_max = ax1.get_xlim()
y_at_xmin1 = b * x1_min + a
y_at_xmax1 = b * x1_max + a
ax1.set_xlim([x1_min, x1_max])
_ = ax1.plot([x1_min, x1_max], [y_at_xmin1, y_at_xmax1], c='green')


# Datos de test
ax2.scatter(df_test['edad'], df_test['presion'], alpha=0.2, edgecolor='black')
ax2.set_ylabel('presion')
ax2.set_xlabel('edad')
ax2.set_title('Datos de validación')

x2_min, x2_max = ax2.get_xlim()
y_at_xmin2 = b * x2_min + a
y_at_xmax2 = b * x2_max + a
ax2.set_xlim([x2_min, x2_max])
_ = ax2.plot([x2_min, x2_max], [y_at_xmin2, y_at_xmax2], c='green');

In [ ]:
plt.scatter(df_test['edad'], df_test['presion'], alpha=0.2, edgecolor='black')
x_sol = np.arange(0, 1.1, .1)
y_sol = a + b * x_sol
plt.plot(x_sol, y_sol, 'r')
plt.xlabel('edad')
plt.ylabel('presion')
plt.title('Distancia a la predicción')
for x, y in zip(df_test['edad'], df_test['presion']):
    y_pred = a + b * x
    plt.plot([x, x], [y, y_pred], 'k', alpha=0.8)

# Predicción de los valores de validación
yhat = a + df_test['edad'] * b
print('ECM por pseudoinversa:', np.mean((yhat - df_test['presion'])**2))

x_min, x_max = plt.xlim()
y_at_xmin = b * x_min + a
y_at_xmax = b * x_max + a
plt.xlim([x_min, x_max])
plt.plot([x_min, x_max], [y_at_xmin, y_at_xmax], c='green');

# Gradiente descendente

<img src="images/descenso_gradiente_ilustracion.jpeg" alt="Drawing" style="width: 600px;"/>



* Inicialización de los parámetros $a$ y $b$
* Definición de los hiperparámetros $\eta$ (constante de aprendizaje) y número de epochs (cantidad de iteraciones de entrenamiento)
* Iniciar loop de pasos del 1 al 4





1. Computar función de costo, ej.: ECM

\begin{equation} 
ECM = \frac{1}{N}\sum_i^N (y_i - \hat{y}_i)^2 \\
ECM = \frac{1}{N}\sum_i^N (y_i - a - bx_i)^2
\end{equation}

2. Computar gradientes

\begin{equation} 
\frac{\partial ECM}{\partial a} = \frac{1}{N}\sum_i^N 2(y_i - a - bx_i)(-1) = -2\frac{1}{N}\sum_i^N (y_i - \hat{y}_i) \\
\frac{\partial ECM}{\partial b} = \frac{1}{N}\sum_i^N 2(y_i - a - bx_i)(-x_i) = -2\frac{1}{N}\sum_i^N x_i(y_i - \hat{y}_i)
\end{equation}

3. Actualizar parámetros

\begin{equation} 
a = a - \eta \frac{\partial ECM}{\partial a}  \\
b = b - \eta \frac{\partial ECM}{\partial b}
\end{equation}

4. Volver al paso 1

### A tener en cuenta

Si tenemos una red con solo 2 parámetros, podemos representar la función de coste en 3 dimensiones (los parámetros en el plano xy, y el coste en el plano z)

<img src="images/curva_ideal.png" alt="Drawing" style="width: 350px;"/>


Pero probablemente vamos a tener millones de parámetros, ademas de tener minimos locales que durante el entrenamiento pueden confundirse con el mínimo global.

<img src="images/curva_real.png" alt="Drawing" style="width: 350px;"/>


### La elección de la tasa de aprendizaje

* El valor no deberia ser ni muy grande (posibles oscilaciones) ni muy chico (lento, mas iteraciones, sobreajuste) 
* Fija o variable en el tiempo.
* La misma para toda la red o varia por capa.

<img src="images/learning_rate.png" alt="Drawing" style="width: 600px;"/>


## Usando numpy

In [ ]:
x_train = df_train['edad'].values
y_train = df_train['presion'].values

x_test = df_test['edad'].values
y_test = df_test['presion'].values

In [ ]:
def calcular_error(y, yhat):
    error = np.sum((y - yhat)**2) / len(y)
    return error

In [ ]:
# Iniciar parámetros a y b
np.random.seed(42)

a = np.random.randn(1)
b = np.random.randn(1)

print('a y b iniciales:', a, b)

lr = 0.0001         # constante de aprendizaje
n_epochs = 1000   # número de iteraciones

for epoch in range(n_epochs):
    # Computar predicción del modelo
    yhat = a + b * x_train
    
    # Calcular error cuadrático medio (ECM)
    error = (y_train - yhat)
    loss = (error ** 2).mean()
    
    # Computar gradiente
    a_grad = -2 * error.mean()
    b_grad = -2 * (x_train * error).mean()
    
    # Actualizar parámetros
    a = a - (lr * a_grad)
    b = b - (lr * b_grad)

# Computar predicción final
yhat = a + b * x_train
# Calcular ECM final
error = (y_train - yhat)
loss = (error ** 2).mean()

print('a y b finales:', a, b)
print('ECM por gradiente descendente:', loss)

## Pytorch


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convertir los arrays de numpy a tensores de pytorch, castearlos a float y 
# alojarlos en el dispositivo disponible
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

print(type(x_train), type(x_train_tensor), x_train_tensor.type())

In [ ]:
# Se puede volver a convertir el tensor de pytorch a numpy
x_train_tensor_numpy = x_train_tensor.numpy()

In [ ]:
# Pero primero hay que alojarlo en cpu
x_train_tensor_numpy = x_train_tensor.to('cpu').numpy()

Los tensores de datos no son como ```x_train``` e ```y_train``` no son variables entrenables, pero sí lo son ```a``` y ```b```, y para esto nos sirve ```requires_grad=True```.

In [ ]:
torch.manual_seed(42)
# Para el algoritmo gradiente descendente necesitamos calcular el gradiente de 
# los parámetros
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
print(a, b)

# En GPU
a = torch.randn(1, dtype=torch.float).to(device)
b = torch.randn(1, dtype=torch.float).to(device)
a.requires_grad_()
b.requires_grad_()
print(a, b)

# Otra opción
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
print(a, b)

## Gradiente descendente con Pytorch

In [ ]:
lr = 1e-1
n_epochs = 1000

torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

print('a y b iniciales:', a, b)

# Elegir el algoritmo de optimización, en este caso Stochastic Gradient Descent
optimizer = optim.SGD([a, b], lr=lr)

for epoch in range(n_epochs):
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()
    
    # el método backward() aplicado a la función de costo calcula los gradientes 
    # de los parámetros
    loss.backward()

    # actualizar parámetros
    optimizer.step()

    # limpiar el cálculo de los gradientes para que no acumule
    optimizer.zero_grad()

print('a y b finales:', a, b)
